In [ ]:
!pip install optuna

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import roc_auc_score
import optuna
import joblib

# Read data
data = pd.read_excel('TrainClass.xlsx')
real_data = pd.read_excel('TestClass.xlsx')

# Prepare training data
Y, X = data['FRACASO'], data.drop(['FRACASO', 'CODIGO_EMPRESA'], axis=1)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

# Prepare testing data
X = real_data.drop(['CODIGO_EMPRESA'], axis=1)

# Define the objective function for BaggingClassifier with RandomForestClassifier
def bagging_rf_objective(trial):
    # Define hyperparameters for RandomForestClassifier
    rf_n_estimators = trial.suggest_int('rf_n_estimators', 50, 200)
    rf_max_depth = trial.suggest_int('rf_max_depth', 1, 10)

    # Define hyperparameters for BaggingClassifier
    bagging_n_estimators = trial.suggest_int('bagging_n_estimators', 10, 100)
    bagging_max_samples = trial.suggest_float('bagging_max_samples', 0.1, 1.0)

    # Create the base classifier with suggested hyperparameters
    base_classifier = RandomForestClassifier(n_estimators=rf_n_estimators, max_depth=rf_max_depth, random_state=0)

    # Create the bagging classifier with the base classifier
    bagging_classifier = BaggingClassifier(
        base_classifier,
        n_estimators=bagging_n_estimators,
        max_samples=bagging_max_samples,
        random_state=0
    )

    # Define cross-validation strategy (StratifiedKFold for classification)
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

    # Perform cross-validation and get mean AUC
    auc_scores = cross_val_score(bagging_classifier, X_train, Y_train, cv=cv, scoring='roc_auc')
    mean_auc = auc_scores.mean()

    return mean_auc

# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize')
study.optimize(bagging_rf_objective, n_trials=100)

# Get the best hyperparameters
best_params = study.best_params
print(f'Best Hyperparameters for BaggingClassifier + RandomForestClassifier: {best_params}')

# Create the base classifier with the best hyperparameters for RandomForestClassifier
rf_classifier = RandomForestClassifier(n_estimators=best_params['rf_n_estimators'], max_depth=best_params['rf_max_depth'], random_state=0)

# Create the bagging classifier with the best hyperparameters
bagging_classifier = BaggingClassifier(
    rf_classifier,
    n_estimators=best_params['bagging_n_estimators'],
    max_samples=best_params['bagging_max_samples'],
    random_state=0
)

# Train the bagging classifier
bagging_classifier.fit(X_train, Y_train)

# Predictions on training set
y_pred_train = bagging_classifier.predict(X_train)

# Calculate AUC for training set
auc_train = roc_auc_score(Y_train, y_pred_train)
print(f'AUC for Training Set: {auc_train:.4f}')

# Predictions on testing set
y_pred_test = bagging_classifier.predict(X_test)

# Calculate AUC for testing set
auc_test = roc_auc_score(Y_test, y_pred_test)
print(f'AUC for Testing Set: {auc_test:.4f}')

# Save the trained model to a file
joblib.dump(bagging_classifier, 'bagging_rf_model.joblib')

# Load model and generate probability CSV
loaded_model = joblib.load('bagging_rf_model.joblib')
prob = loaded_model.predict_proba(X)[:, 1]
prob_df = pd.DataFrame(prob, columns=['Probability'])
prob_df.index = prob_df.index + 1
prob_df.index.name = 'Id'
prob_df.to_csv('intento.csv', index=True)


[I 2023-12-05 15:37:34,295] A new study created in memory with name: no-name-d449429b-d87b-4197-b0a3-50a3e1f53e33
[I 2023-12-05 15:38:15,402] Trial 0 finished with value: 0.8568078612209046 and parameters: {'rf_n_estimators': 69, 'rf_max_depth': 3, 'bagging_n_estimators': 52, 'bagging_max_samples': 0.31773332948538036}. Best is trial 0 with value: 0.8568078612209046.
[I 2023-12-05 15:38:59,349] Trial 1 finished with value: 0.8600832235397451 and parameters: {'rf_n_estimators': 68, 'rf_max_depth': 3, 'bagging_n_estimators': 62, 'bagging_max_samples': 0.19768695688447513}. Best is trial 1 with value: 0.8600832235397451.
[I 2023-12-05 15:40:12,893] Trial 2 finished with value: 0.8417246376811593 and parameters: {'rf_n_estimators': 134, 'rf_max_depth': 8, 'bagging_n_estimators': 25, 'bagging_max_samples': 0.9657863176419709}. Best is trial 1 with value: 0.8600832235397451.
[I 2023-12-05 15:41:10,522] Trial 3 finished with value: 0.8604451032059728 and parameters: {'rf_n_estimators': 89, 'r

Best Hyperparameters for BaggingClassifier + RandomForestClassifier: {'rf_n_estimators': 159, 'rf_max_depth': 1, 'bagging_n_estimators': 87, 'bagging_max_samples': 0.10050689805404925}
AUC for Training Set: 0.5000
AUC for Testing Set: 0.5000
